In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
import time
import os
import random
import math

In [2]:
with open("../4_power/data/proposals_20221007.json") as file:
    df = json.load(file)


In [3]:
# for i in range(0,int(np.round(len(snapshot)/8000))):
#     path = "../4_power/data/proposals_20221007_" + str(i) + ".json"
#     i = i * 8000
#     df = pd.DataFrame(snapshot[i:i+8000])
    
#     df.to_json(path)'

In [14]:
def strategy_tags (strategy):
    tags = set()

    # dict keys
    for current_dict in strategy:
        if current_dict['params'] != None:
            tags = tags | set(current_dict['params'].keys())

    return tags

In [5]:
df = pd.DataFrame(df)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69801 entries, 0 to 69800
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  69801 non-null  object 
 1   created             69801 non-null  int64  
 2   space               69801 non-null  object 
 3   network             69801 non-null  object 
 4   symbol              69801 non-null  object 
 5   type                69801 non-null  object 
 6   strategies          69801 non-null  object 
 7   title               69801 non-null  object 
 8   body                69801 non-null  object 
 9   choices             69801 non-null  object 
 10  start               69801 non-null  int64  
 11  end                 69801 non-null  int64  
 12  quorum              69801 non-null  float64
 13  snapshot            69801 non-null  object 
 14  state               69801 non-null  object 
 15  link                69801 non-null  object 
 16  app 

One example

In [8]:
df.iloc[12345,:]['strategies']

[{'params': {'symbol': 'TCR',
   'address': '0xa84918f3280d488eb3369cb713ec53ce386b6cba',
   'decimals': 18,
   'methodABI': {'name': 'getStaked',
    'type': 'function',
    'inputs': [{'name': 'account',
      'type': 'address',
      'internalType': 'address'}],
    'outputs': [{'name': '', 'type': 'uint96', 'internalType': 'uint96'}],
    'constant': True,
    'stateMutability': 'view'}}}]

In [26]:
len(df.iloc[12345,:]['strategies']), strategy_tags(df.iloc[12345,:]['strategies'])

(1, {'address', 'decimals', 'methodABI', 'symbol'})

Another example

In [24]:
df.iloc[22222,:]['strategies']

[{'params': {'symbol': 'aAAVE',
   'address': '0xffc97d72e13e01096502cb8eb52dee56f74dad7b',
   'decimals': 18}},
 {'params': {'graphs': {'137': 'https://api.thegraph.com/subgraphs/name/sameepsi/maticblocks'},
   'symbol': 'Matic AAVE+amAAVE',
   'strategies': [{'name': 'erc20-balance-of',
     'params': {'symbol': 'AAVE',
      'address': '0xD6DF932A45C0f255f85145f286eA0b292B21C90B',
      'decimals': 18},
     'network': '137'},
    {'name': 'erc20-balance-of',
     'params': {'symbol': 'amAAVE',
      'address': '0x1d2a0E5EC8E5bBDCA5CB219e649B565d8e5c3360',
      'decimals': 18},
     'network': 137}]}}]

In [27]:
len(df.iloc[22222,:]['strategies']), strategy_tags(df.iloc[22222,:]['strategies'])

(2, {'address', 'decimals', 'graphs', 'strategies', 'symbol'})

Another example

BTW, a weird token:
https://etherscan.io/token/0xc300d17C49Fbf1E898C5aC66c05A5421c3997f2d

In [11]:
df.iloc[54321,:]['strategies']

[{'params': {'symbol': 'Oong',
   'address': '0xc300d17C49Fbf1E898C5aC66c05A5421c3997f2d',
   'decimals': 18}}]

In [28]:
len(df.iloc[54321,:]['strategies']), strategy_tags(df.iloc[54321,:]['strategies'])

(1, {'address', 'decimals', 'symbol'})

An idea how many will be left if we conditinon on the 'erc20-balance-of' strategy

In [34]:
df['strategy_count'] = df['strategies'].apply(len)
df['strategy_tags'] = df['strategies'].apply(strategy_tags)
df['strategy_tags_str'] = df['strategy_tags'].apply(str)

In [35]:
df[['strategy_count','strategy_tags_str','id']].groupby(['strategy_count','strategy_tags_str']).count().reset_index().sort_values(by='id',ascending=False).head(20)

,strategy_count,strategy_tags_str,id
51,1,"{'symbol', 'address', 'decimals'}",28820
67,1,"{'symbol', 'address'}",5014
1,1,set(),3063
272,2,"{'symbol', 'decimals', 'address'}",2798
177,1,{'symbol'},2601
6,1,"{'address', 'chefAddresses'}",2265
720,6,"{'multiplier', 'strategies', 'graphs', 'symbol...",1749
44,1,"{'symbol', 'address', 'decimals', 'methodABI'}",1586
68,1,"{'symbol', 'addresses'}",1151
290,2,"{'symbol', 'decimals', 'strategies', 'address'}",966
